## iniciando uma sessao no spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import col, when, rand, expr, udf
from random import choice

# Etapa 1: Criar SparkSession e carregar o arquivo
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Exercicio Intro") \
    .getOrCreate()

## lendo ao arquivo "nome_aleatorios.txt" e mostrando as 5 primeiras linhas

In [2]:
df_nomes = spark.read.csv("nomes_aleatorios.txt", header=False, inferSchema=True)
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



## renomeando e mostrando as 10 primeiras linhas 


In [3]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nome")
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nome: string (nullable = true)

+-----------------+
|             Nome|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



## Etapa 3: Adicionar coluna "Escolaridade"

In [4]:
# Etapa 3: Adicionar coluna "Escolaridade"
df_nomes = df_nomes.withColumn(
    "Escolaridade", 
    when(rand() < 0.33, "Fundamental")
    .when((rand() >= 0.33) & (rand() < 0.66), "Medio")
    .otherwise("Superior")
)

df_nomes.show(10)

+-----------------+------------+
|             Nome|Escolaridade|
+-----------------+------------+
|   Frances Bennet|    Superior|
|    Jamie Russell| Fundamental|
|   Edward Kistler|       Medio|
|    Sheila Maurer|    Superior|
| Donald Golightly|       Medio|
|       David Gray|       Medio|
|      Joy Bennett|       Medio|
|      Paul Kriese|    Superior|
|Berniece Ornellas|    Superior|
|    Brian Farrell| Fundamental|
+-----------------+------------+
only showing top 10 rows



## criando uma lista de países e diacionando a coluna

In [5]:
# Lista de países
paises = ["Argentina", "Brasil", "Chile", "Colômbia", "Equador", "Guiana", "Paraguai", "Peru", "Suriname", "Uruguai", "Venezuela", "Bolívia", "Guiana Francesa"]

# Criar uma coluna com o array de países e selecionar um valor aleatório
df_nomes = df_nomes.withColumn("Pais", F.array(*[F.lit(pais) for pais in paises]))

# Selecionar um país aleatório da lista usando o índice aleatório
df_nomes = df_nomes.withColumn("Pais", F.expr("Pais[cast(rand() * size(Pais) as int)]"))

df_nomes = df_nomes.drop("random_index")

df_nomes.show(10)

+-----------------+------------+---------+
|             Nome|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet|    Superior|   Brasil|
|    Jamie Russell| Fundamental| Suriname|
|   Edward Kistler|       Medio|Argentina|
|    Sheila Maurer|    Superior|  Equador|
| Donald Golightly|       Medio|Venezuela|
|       David Gray|       Medio|Argentina|
|      Joy Bennett|       Medio| Suriname|
|      Paul Kriese|    Superior| Suriname|
|Berniece Ornellas|    Superior|Venezuela|
|    Brian Farrell| Fundamental|Venezuela|
+-----------------+------------+---------+
only showing top 10 rows



## Etapa 5: Adicionar coluna "AnoNascimento" com valores aleatórios

In [6]:

df_nomes = df_nomes.withColumn("AnoNascimento", (rand() * (2010 - 1945) + 1945).cast("int"))
df_nomes.show(10)

+-----------------+------------+---------+-------------+
|             Nome|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet|    Superior|   Brasil|         1993|
|    Jamie Russell| Fundamental| Suriname|         1986|
|   Edward Kistler|       Medio|Argentina|         1979|
|    Sheila Maurer|    Superior|  Equador|         1946|
| Donald Golightly|       Medio|Venezuela|         2006|
|       David Gray|       Medio|Argentina|         1986|
|      Joy Bennett|       Medio| Suriname|         1967|
|      Paul Kriese|    Superior| Suriname|         2009|
|Berniece Ornellas|    Superior|Venezuela|         1998|
|    Brian Farrell| Fundamental|Venezuela|         1956|
+-----------------+------------+---------+-------------+
only showing top 10 rows



##  Etapa 6: Selecionar pessoas nascidas no século 21

In [7]:

df_select = df_nomes.filter(df_nomes.AnoNascimento >= 2000)
df_select.show(10)


+--------------------+------------+---------------+-------------+
|                Nome|Escolaridade|           Pais|AnoNascimento|
+--------------------+------------+---------------+-------------+
|    Donald Golightly|       Medio|      Venezuela|         2006|
|         Paul Kriese|    Superior|       Suriname|         2009|
|       Tracy Herring| Fundamental|Guiana Francesa|         2003|
|      Gabriel Colyer|    Superior|        Uruguai|         2004|
|          Daryl Page|    Superior|           Peru|         2002|
|      Katrina Graham|    Superior|Guiana Francesa|         2002|
|          John Meyer| Fundamental|        Bolívia|         2008|
|          Ned Tester|       Medio|      Argentina|         2008|
|Christopher Williams|    Superior|       Colômbia|         2005|
|        Juliet Liles|       Medio|       Paraguai|         2007|
+--------------------+------------+---------------+-------------+
only showing top 10 rows



##  Etapa 7: Usar Spark SQL para selecionar pessoas nascidas no século 21

In [8]:

df_nomes.createOrReplaceTempView("pessoas")

# SQL para selecionar pessoas nascidas a partir de 2000
spark.sql("SELECT * FROM pessoas WHERE AnoNascimento >= 2000").show(10)

+--------------------+------------+---------------+-------------+
|                Nome|Escolaridade|           Pais|AnoNascimento|
+--------------------+------------+---------------+-------------+
|    Donald Golightly|       Medio|      Venezuela|         2006|
|         Paul Kriese|    Superior|       Suriname|         2009|
|       Tracy Herring| Fundamental|Guiana Francesa|         2003|
|      Gabriel Colyer|    Superior|        Uruguai|         2004|
|          Daryl Page|    Superior|           Peru|         2002|
|      Katrina Graham|    Superior|Guiana Francesa|         2002|
|          John Meyer| Fundamental|        Bolívia|         2008|
|          Ned Tester|       Medio|      Argentina|         2008|
|Christopher Williams|    Superior|       Colômbia|         2005|
|        Juliet Liles|       Medio|       Paraguai|         2007|
+--------------------+------------+---------------+-------------+
only showing top 10 rows



## Etapa 8: Contar Millenials

In [9]:

millennials_count = df_nomes.filter((df_nomes.AnoNascimento >= 1980) & (df_nomes.AnoNascimento <= 1994)).count()
print(f"Número de Millenials: {millennials_count}")

Número de Millenials: 2306829


## Etapa 9: Contar Millenials usando Spark SQL

In [10]:

spark.sql("SELECT COUNT(*) FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994").show()


+--------+
|count(1)|
+--------+
| 2306829|
+--------+



## Etapa 10: Contar pessoas de cada geração por país

In [11]:

geracoes_sql = """
SELECT Pais, 
       CASE
           WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
           WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geracao X'
           WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
           WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geracao Z'
       END AS Geracao, 
       COUNT(*) AS Quantidade
FROM pessoas
GROUP BY Pais, Geracao
ORDER BY Pais, Geracao
"""


df_geracoes = spark.sql(geracoes_sql)
df_geracoes.show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    237612|
|Argentina|   Geracao X|    177413|
|Argentina|   Geracao Z|    177942|
|Argentina| Millennials|    176866|
|  Bolívia|Baby Boomers|    236871|
|  Bolívia|   Geracao X|    177672|
|  Bolívia|   Geracao Z|    177049|
|  Bolívia| Millennials|    178320|
|   Brasil|Baby Boomers|    236032|
|   Brasil|   Geracao X|    177938|
|   Brasil|   Geracao Z|    177724|
|   Brasil| Millennials|    177519|
|    Chile|Baby Boomers|    237151|
|    Chile|   Geracao X|    177665|
|    Chile|   Geracao Z|    177503|
|    Chile| Millennials|    177923|
| Colômbia|Baby Boomers|    235950|
| Colômbia|   Geracao X|    177157|
| Colômbia|   Geracao Z|    177942|
| Colômbia| Millennials|    176635|
+---------+------------+----------+
only showing top 20 rows

